In [8]:
import pandas as pd
df = pd.read_csv('keras.csv',sep=';')

In [11]:
import re
import pymorphy2

ma = pymorphy2.MorphAnalyzer()

def clean_text(text):
    try:
     text = text.replace("\\", " ").replace(u"╚", " ").replace(u"╩", " ")
     text = text.lower()
     text = re.sub('\-\s\r\n\s{1,}|\-\s\r\n|\r\n', '', text) #deleting newlines and line-breaks
     text = re.sub('[.,:;_%©?*,!@#$%^&()\d]|[+=]|[[]|[]]|[/]|"|\s{2,}|-', ' ', text) #deleting symbols  
     text = " ".join(ma.parse(str(word))[0].normal_form for word in text.split())
     text = ' '.join(word for word in text.split() if len(word)>3)
     text = text.encode("utf-8")
    except:
     print(text)

    return text

df['Description'] = df.apply(lambda x: clean_text(x['Post_clean']), axis=1)


nan


In [2]:
df

,TNVED,OPISANIE,DATA,Post_clean,TNVED2
0,8471,УСТРОЙСТВА ВЫЧИСЛИТЕЛЬНЫХ МАШИН - ОТЛАДОЧНАЯ П...,04.10.2021,устройства вычислительных машин отладочная пла...,84
1,6212,"БЮСТГАЛЬТЕРЫ ЖЕНСКИЕ. РОСТ ДО 185 СМ,ОБХВАТ ГР...",20.09.2021,бюстгальтеры женские рост 185 см обхват груди ...,62
2,8548,ЭЛЕКТРИЧЕСКИЕ ЧАСТИ ОБОРУДОВАНИЯ:,24.09.2021,электрические части оборудования,85
3,4403,"ПИЛОВОЧНИК ОСИНОВЫЙ (ТОПОЛЬ ДРОЖАЩИЙ (ОСИНА) ""...",12.10.2021,пиловочник осиновый тополь дрожащий осина popu...,44
4,9507,ПРИНАДЛЕЖНОСТИ ДЛЯ РЫБНОЙ ЛОВЛИ: БЛЕСНА ВРАЩАЮ...,17.09.2021,принадлежности рыбной ловли блесна вращающаяся...,95
...,...,...,...,...,...
201727,4016,КОЛЬЦО УПЛОТНИТЕЛЬНОЕ ИЗ НЕПОРИСТОЙ ВУЛКАНИЗИР...,16.09.2021,кольцо уплотнительное непористой вулканизирова...,40
201728,8512,"КОМПОНЕНТЫ, ПОСТАВЛЯЕМЫЕ В КАЧЕСТВЕ СМЕННЫХ (З...",13.10.2021,компоненты поставляемые качестве сменных запас...,85
201729,7326,ИЗДЕЛИЯ ИЗ ЧЕРНЫХ МЕТАЛЛОВ КОМБИНИРОВАННОЙ ОБР...,12.10.2021,изделия черных металлов комбинированной обрабо...,73
201730,8708,ЧАСТИ И ПРИНАДЛЕЖНОСТИ КУЗОВОВ ДЛЯ ПРОМ. СБОРК...,20.10.2021,части принадлежности кузовов пром сборки мотор...,87


In [3]:
categories = {}
for key,value in enumerate(df['TNVED2'].unique()):
    categories[value] = key + 1

# Запишем в новую колонку числовое обозначение категории 
df['category_code'] = df['TNVED2'].map(categories)

total_categories = len(df['TNVED2'].unique()) + 1
print('Всего категорий: {}'.format(total_categories))

Всего категорий: 64


In [4]:
df.to_pickle('dataframe_ver_1.pkl')

In [5]:
df = pd.read_pickle('dataframe_ver_1.pkl')

In [6]:
df = df.sample(frac=1).reset_index(drop=True)

In [7]:
descriptions = df['Description']
categories = df[u'category_code']

# Посчитаем максимальную длинну текста описания в словах
max_words = 0
for desc in descriptions:
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальная длина описания: {} слов'.format(max_words))

maxSequenceLength = 55

AttributeError: 'float' object has no attribute 'split'

In [ ]:
from keras.preprocessing.text import Tokenizer

# создаем единый словарь (слово -> число) для преобразования
tokenizer = Tokenizer()
tokenizer.fit_on_texts(descriptions.tolist())

# Преобразуем все описания в числовые последовательности, заменяя слова на числа по словарю.
textSequences = tokenizer.texts_to_sequences(descriptions.tolist())

In [ ]:
def load_data_from_arrays(strings, labels, train_test_split=0.9):
    data_size = len(strings)
    test_size = int(data_size - round(data_size * train_test_split))
    print("Test size: {}".format(test_size))
    
    print("\nTraining set:")
    x_train = strings[test_size:]
    print("\t - x_train: {}".format(len(x_train)))
    y_train = labels[test_size:]
    print("\t - y_train: {}".format(len(y_train)))
    
    print("\nTesting set:")
    x_test = strings[:test_size]
    print("\t - x_test: {}".format(len(x_test)))
    y_test = labels[:test_size]
    print("\t - y_test: {}".format(len(y_test)))

    return x_train, y_train, x_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_data_from_arrays(textSequences, categories, train_test_split=0.8)

In [ ]:
total_words = len(tokenizer.word_index)
print('В словаре {} слов'.format(total_words))

In [ ]:
num_words = 1000

print(u'Преобразуем описания заявок в векторы чисел...')
tokenizer = Tokenizer(num_words=num_words)
X_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
X_test = tokenizer.sequences_to_matrix(X_test, mode='binary')
print('Размерность X_train:', X_train.shape)
print('Размерность X_test:', X_test.shape)

print(u'Преобразуем категории в матрицу двоичных чисел '
      u'(для использования categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout

# количество эпох\итераций для обучения
epochs = 10

print(u'Собираем модель...')
model = Sequential()
model.add(Dense(512, input_shape=(num_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(total_categories))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=epochs,
                    verbose=1,

In [ ]:
score = model.evaluate(X_test, y_test,
                       batch_size=32, verbose=1)
print()
print(u'Оценка теста: {}'.format(score[0]))
print(u'Оценка точности модели: {}'.format(score[1]))

In [ ]:
import matplotlib.pyplot as plt

# График точности модели
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# График оценки loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
from keras.preprocessing.text import Tokenizer

# создаем единый словарь (слово -> число) для преобразования
tokenizer = Tokenizer()
tokenizer.fit_on_texts(descriptions.tolist())

# Преобразуем все описания в числовые последовательности, заменяя слова на числа по словарю.
textSequences = tokenizer.texts_to_sequences(descriptions.tolist())

In [ ]:
X-train, y_train, X_test, y_test = load_data_from_arrays(textSequences, categories, train_test_split=0.8)
# Максимальное количество слов в самом длинном описании заявки
max_words = 0
for desc in descriptions.tolist():
    words = len(desc.split())
    if words > max_words:
        max_words = words
print('Максимальное количество слов в самом длинном описании заявки: {} слов'.format(max_words))

total_unique_words = len(tokenizer.word_counts)
print('Всего уникальных слов в словаре: {}'.format(total_unique_words))

maxSequenceLength = max_words

In [ ]:
vocab_size = round(total_unique_words/10)

In [ ]:

tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(descriptions)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = sequence.pad_sequences(X_train, maxlen=maxSequenceLength)
X_test = sequence.pad_sequences(X_test, maxlen=maxSequenceLength)

print('Размерность X_train:', X_train.shape)
print('Размерность X_test:', X_test.shape)

print(u'Преобразуем категории в матрицу двоичных чисел '
      u'(для использования categorical_crossentropy)')
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

In [ ]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(y_train)
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

num_classes = np.max(y_train) + 1
print('Количество категорий для классификации: {}'.format(num_classes))

In [ ]:
from  keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

# максимальное количество слов для анализа
max_features = vocab_size

print(u'Собираем модель...')
model = Sequential()
model.add(Embedding(max_features, maxSequenceLength))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print (model.summary())


In [ ]:
batch_size = 32
epochs = 3

print(u'Тренируем модель...')
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(X_test, y_test))


In [ ]:
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
print()
print(u'Оценка теста: {}'.format(score[0]))
print(u'Оценка точности модели: {}'.format(score[1]))

In [ ]:
imoort matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_acc"], label="val_acc")
plt.title("Эффективность обучения")
plt.xlabel("Повторения #")
plt.ylabel("Ошибки")
plt.legend(loc="lower left")